In [2]:
import os
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score
from geopandas import read_file
import xarray as xr
import rasterio
import csv

dir_data = "/storage/scratch/users/ch21o450/data/LandClim_Output/"
dir_validation = "/storage/homefs/ch21o450/data/SpeciesData2/"

In [12]:
for time in times:
    time[0]

TypeError: 'int' object is not subscriptable

In [23]:
# Set up parameters
models = ["GAM", "GBM"]
taxas = ["Mammals"]
time = [35]
time = time[0]
years = ["1845", "1990", "1995", "2009", "2010", "2020", "2026", "2032", "2048", "2050",
         "2052", "2056", "2080", "2100", "2150", "2200", "2250"]
average_auc_scores = []  # List to store average AUC scores
results = []  # List to store results

if time == 35 or time == 65:
    model_names = ['GFDL-ESM2M', 'IPSL-CM5A-LR', 'HadGEM2-ES', 'MIROC5']
    scenarios = ["rcp26", "rcp60"]
elif time == 85:
    model_names = ['IPSL-CM5A-LR', 'HadGEM2-ES', 'MIROC5']
    scenarios = ["rcp26"]

for taxa in taxas:
    for model in models:
        for model_name in model_names:
            dir_species = "/storage/scratch/users/ch21o450/data/LandClim_Output/" + model + "/" + taxa + "/EWEMBI/"
            available_file = os.listdir(dir_species)
            available_names = [x.split("_[1146].nc")[0] for x in available_file]
            species_names = available_names[:1]
            
            dir_species = "/storage/workspaces/wa_climate/climate_trt/data/BioScen15/individual_projections/" + taxa+ "_" + model +"_results_climate/"
            available_file = os.listdir(dir_species)
            available_names = [x.split(".csv")[0] for x in available_file]

            formatted_names = []

            for species_name in available_names[:8]:
                split_species_name = species_name.split("_")[:2]
                formatted_species_name = " ".join(split_species_name)
                formatted_names.append(formatted_species_name)

            for i, species_name in enumerate(species_names):
                formatted_species_name = species_name.replace(" ", "_")

                for file_name in available_file:
                    if formatted_species_name in file_name and model + '_dispersal.csv.xz' in file_name:
                        species_file = file_name
                        species_file2 = [x.split(".csv")[0] for x in species_file] 
                        break
                else:
                    bioscen_species = None
                    continue

                bioscen_species = pd.read_csv(dir_species + file_name)

                available_files_iucn = formatted_species_name + ".csv"


                lon = bioscen_species["x"]
                lat = bioscen_species["y"]
                z = bioscen_species["presence"]

                df = pd.DataFrame({"lon": lon, "lat": lat, "vals": z})

In [27]:
bioscen_species

,x,y,GFDL.ESM2M_piControl_1845,HadGEM2.ES_piControl_1845,IPSL.CM5A-LR_piControl_1845,MIROC5_piControl_1845,GFDL.ESM2M_historical_1990,HadGEM2.ES_historical_1990,IPSL.CM5A-LR_historical_1990,MIROC5_historical_1990,...,HadGEM2.ES_rcp26_2250,IPSL.CM5A-LR_rcp26_2250,MIROC5_rcp26_2250,presence,dispersal4,dispersal3,dispersal2,dispersal1,fulldisp,areaKM2
0,-178.25,-14.25,0.159867,0.171467,0.279194,0.172161,0.147804,0.145932,0.167806,0.212452,...,0.151566,0.182201,0.147464,NaN,NaN,NaN,NaN,NaN,1,2983.188554
1,-177.25,28.25,0.019551,0.267213,0.018077,0.327666,0.017587,0.019148,0.020384,0.016781,...,0.012453,0.016484,0.013334,NaN,NaN,NaN,NaN,NaN,1,2712.770667
2,-176.75,-44.25,0.026211,0.026753,0.020176,0.028573,0.028989,0.028679,0.019212,0.028817,...,0.029489,0.018705,0.023659,NaN,NaN,NaN,NaN,NaN,1,2207.851480
3,-176.75,-43.75,0.026594,0.027227,0.019557,0.028963,0.029228,0.029073,0.018872,0.028898,...,0.029425,0.018587,0.021648,NaN,NaN,NaN,NaN,NaN,1,2226.471245
4,-176.25,-43.75,0.026741,0.028091,0.019496,0.028915,0.029288,0.029144,0.018832,0.029357,...,0.029488,0.018521,0.021088,NaN,NaN,NaN,NaN,NaN,1,2226.471245
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13553,179.25,-16.25,0.259884,0.314172,0.391615,0.313063,0.277113,0.395547,0.367677,0.368476,...,0.332698,0.293319,0.311406,NaN,NaN,NaN,NaN,NaN,1,2955.105366
13554,179.25,-8.75,0.137966,0.124002,0.138066,0.126007,0.137671,0.123331,0.137617,0.123263,...,0.101963,0.101809,0.101243,NaN,NaN,NaN,NaN,NaN,1,3041.689436
13555,179.25,-8.25,0.137092,0.126861,0.138378,0.137863,0.138188,0.123830,0.138206,0.123653,...,0.102282,0.101734,0.101492,NaN,NaN,NaN,NaN,NaN,1,3045.633050
13556,179.75,-16.75,0.166130,0.269146,0.400671,0.311771,0.244987,0.399613,0.380272,0.379968,...,0.341478,0.334276,0.371429,NaN,NaN,NaN,NaN,NaN,1,2947.523248


In [9]:
dir_species

'/storage/scratch/users/ch21o450/data/LandClim_Output/GBM/Mammals/EWEMBI/'

In [8]:
formatted_species_name

'Corynorhinus_rafinesquii'

In [7]:
# Set up parameters
models = ["GAM", "GBM"]
taxas = ["Mammals"]
time = [35]
time = time[0]
years = ["1845", "1990", "1995", "2009", "2010", "2020", "2026", "2032", "2048", "2050",
         "2052", "2056", "2080", "2100", "2150", "2200", "2250"]
average_auc_scores = []  # List to store average AUC scores
results = []  # List to store results

if time == 35 or time == 65:
    model_names = ['GFDL-ESM2M', 'IPSL-CM5A-LR', 'HadGEM2-ES', 'MIROC5']
    scenarios = ["rcp26", "rcp60"]
elif time == 85:
    model_names = ['IPSL-CM5A-LR', 'HadGEM2-ES', 'MIROC5']
    scenarios = ["rcp26"]

for taxa in taxas:
    for model in models:
        for model_name in model_names:
            dir_species = "/storage/scratch/users/ch21o450/data/LandClim_Output/" + model + "/" + taxa + "/EWEMBI/"
            available_file = os.listdir(dir_species)
            available_names = [x.split("_[1146].nc")[0] for x in available_file]
            species_names = available_names[:10]
            

            for i, species_name in enumerate(species_names):
                formatted_species_name = species_name.replace(" ", "_")

                for file_name in available_file:
                    if formatted_species_name in file_name and model + '_dispersal.csv.xz' in file_name:
                        species_file = file_name
                        species_file2 = [x.split(".csv")[0] for x in species_file] 
                        break
                else:
                    bioscen_species = None
                    continue

                bioscen_species = pd.read_csv(dir_species + file_name)

                available_files_iucn = formatted_species_name + ".csv"
                if available_files_iucn in os.listdir(dir_habclass):
                    IUCN = pd.read_csv(dir_habclass + available_files_iucn)
                else:
                    continue

                lon = bioscen_species["x"]
                lat = bioscen_species["y"]
                z = bioscen_species[bioscen_model_name + '_' + selected_year]

                df = pd.DataFrame({"lon": lon, "lat": lat, "vals": z})
                        
            for scenario in scenarios:
                auc_scores = []  # List to store AUC scores

                for species_name in species_names:
                    # Load species distribution model output
                    sd_model_file = f"{dir_data}/{model}/{taxa}/{model_name}/{scenario}/{species_name}_[{time}].nc"
                    sd_model_data = xr.open_dataset(sd_model_file, decode_times=False)
                    sd_model_data = sd_model_data.isel(time=0)
                    sd_model_data = sd_model_data['sum_bin']  # Select the 'sum_bin' variable or 'newvalue'

                    # Convert probability of occurrence to binary presence/absence data using a threshold value
                    threshold = 0.05
                    sd_model_binary = np.where(sd_model_data > threshold, 1, 0)

                    try:
                        
                        
                    for i, species_name in enumerate(formatted_names):
                        formatted_species_name = species_name.replace(" ", "_")

                        for file_name in available_file:
                            if formatted_species_name in file_name and model + '_dispersal.csv.xz' in file_name:
                                species_file = file_name
                                species_file2 = [x.split(".csv")[0] for x in species_file] 
                                break
                        else:
                            bioscen_species = None
                            continue

                        bioscen_species = pd.read_csv(dir_species + file_name)

                        available_files_iucn = formatted_species_name + ".csv"
                        if available_files_iucn in os.listdir(dir_habclass):
                            IUCN = pd.read_csv(dir_habclass + available_files_iucn)
                        else:
                            continue

                        lon = bioscen_species["x"]
                        lat = bioscen_species["y"]
                        z = bioscen_species[bioscen_model_name + '_' + selected_year]

                        df = pd.DataFrame({"lon": lon, "lat": lat, "vals": z})

                        validation_file = [f for f in os.listdir(dir_validation) if species_name in f][0]
                        validation_file_path = os.path.join(dir_validation, validation_file)

                        import imageio

                        # Read the TIFF file
                        validation_data = imageio.imread(validation_file_path)

                        # Convert the data to a NumPy array
                        validation_data = np.array(validation_data)

                        # Create a mask for non-null values
                        nodata_mask = validation_data != 0

                        # Flatten the validation data array and extract only the non-null values
                        validation_points_masked = validation_data.flatten()[nodata_mask.flatten()]
                        validation_points_binary = np.where(validation_points_masked > 0, 1, 0)

                        # Flatten the model data array and extract only the corresponding non-null values
                        model_points_masked = sd_model_binary.flatten()[nodata_mask.flatten()]

                        auc_score = roc_auc_score(validation_points_binary, model_points_masked)
                        auc_scores.append(auc_score)
                    except IndexError:
                        continue

                average_auc = np.mean(auc_scores)
                average_auc_scores.append(average_auc)

                result = {
                "Taxa": taxa,
                "Model": model,
                "Time": time,
                "Model Name": model_name,
                "Scenario": scenario,
                "AUC": average_auc
                }
                results.append(result)

        csv_file = "/storage/homefs/ch21o450/scripts/BioScenComb/validation/averaged_GCM_auc_scores_" + str(time) + "_" + model + "_" + taxa + "_"   +  ".csv"

        with open(csv_file, mode="w", newline="") as file:
            fieldnames = ["Taxa", "Model", "Time", "Model Name", "Scenario", "AUC"]
            writer = csv.DictWriter(file, fieldnames=fieldnames)
            writer.writeheader()
            writer.writerows(results)

        print("Results have been written to", csv_file)





IndentationError: expected an indented block (3532332916.py, line 70)

In [3]:
results

[{'Taxa': 'Mammals',
  'Model': 'GAM',
  'Time': 35,
  'Model Name': 'GFDL-ESM2M',
  'Scenario': 'rcp26',
  'AUC': 0.8318912814436802},
 {'Taxa': 'Mammals',
  'Model': 'GAM',
  'Time': 35,
  'Model Name': 'GFDL-ESM2M',
  'Scenario': 'rcp60',
  'AUC': 0.8343794114160874},
 {'Taxa': 'Mammals',
  'Model': 'GAM',
  'Time': 35,
  'Model Name': 'IPSL-CM5A-LR',
  'Scenario': 'rcp26',
  'AUC': 0.829654438696158},
 {'Taxa': 'Mammals',
  'Model': 'GAM',
  'Time': 35,
  'Model Name': 'IPSL-CM5A-LR',
  'Scenario': 'rcp60',
  'AUC': 0.8355269642875012},
 {'Taxa': 'Mammals',
  'Model': 'GAM',
  'Time': 35,
  'Model Name': 'HadGEM2-ES',
  'Scenario': 'rcp26',
  'AUC': 0.8363312273651615},
 {'Taxa': 'Mammals',
  'Model': 'GAM',
  'Time': 35,
  'Model Name': 'HadGEM2-ES',
  'Scenario': 'rcp60',
  'AUC': 0.8358901750132086},
 {'Taxa': 'Mammals',
  'Model': 'GAM',
  'Time': 35,
  'Model Name': 'MIROC5',
  'Scenario': 'rcp26',
  'AUC': 0.8351397592893036},
 {'Taxa': 'Mammals',
  'Model': 'GAM',
  'Time': 

In [ ]:
# assess the average model performance for each combination of taxa, time, and model_name
# Set up parameters
import csv


# Set up parameters
models = ["GAM", "GBM"]
taxas = ["Mammals"]
times = [35, 65, 85]
years = ["1845", "1990", "1995", "2009", "2010", "2020", "2026", "2032", "2048", "2050",
         "2052", "2056", "2080", "2100", "2150", "2200", "2250"]

results = []

for taxa in taxas:
    for model in models:
        dir_species = "/storage/scratch/users/ch21o450/data/LandClim_Output/" + model + "/" + taxa + "/EWEMBI/"
        available_file = os.listdir(dir_species)
        available_names = [x.split("_[1146].nc")[0] for x in available_file]
        species_names = available_names[:10]

        for time in times:
            if time == 35 or time == 65:
                model_names = ['GFDL-ESM2M', 'IPSL-CM5A-LR', 'HadGEM2-ES', 'MIROC5']
                scenarios = ["rcp26", "rcp60"]
            elif time == 85:
                model_names = ['IPSL-CM5A-LR', 'HadGEM2-ES', 'MIROC5']
                scenarios = ["rcp26"]

            for model_name in model_names:
                for scenario in scenarios:
                    auc_scores = []  # List to store AUC scores

                    for species_name in species_names:
                        # Load species distribution model output
                        sd_model_file = f"{dir_data}/{model}/{taxa}/{model_name}/{scenario}/{species_name}_[{time}].nc"
                        sd_model_data = xr.open_dataset(sd_model_file, decode_times=False)
                        sd_model_data = sd_model_data.isel(time=0)
                        sd_model_data = sd_model_data['sum_bin']  # Select the 'sum_bin' variable or 'newvalue'

                        # Convert probability of occurrence to binary presence/absence data using a threshold value
                        threshold = 0.05
                        sd_model_binary = np.where(sd_model_data > threshold, 1, 0)

                        try:
                            validation_file = [f for f in os.listdir(dir_validation) if species_name in f][0]
                            validation_file_path = os.path.join(dir_validation, validation_file)

                            import imageio

                            # Read the TIFF file
                            validation_data = imageio.imread(validation_file_path)

                            # Convert the data to a NumPy array
                            validation_data = np.array(validation_data)

                            # Create a mask for non-null values
                            nodata_mask = validation_data != 0

                            # Flatten the validation data array and extract only the non-null values
                            validation_points_masked = validation_data.flatten()[nodata_mask.flatten()]
                            validation_points_binary = np.where(validation_points_masked > 0, 1, 0)

                            # Flatten the model data array and extract only the corresponding non-null values
                            model_points_masked = sd_model_binary.flatten()[nodata_mask.flatten()]

                            auc_score = roc_auc_score(validation_points_binary, model_points_masked)
                            auc_scores.append(auc_score)
                        except IndexError:
                            continue

                        average_auc = np.mean(auc_scores)
                        result = {
                            "Taxa": taxa,
                            "Model": model,                       
                            "Time": time,
                            "Model Name": model_name,
                            "Scenario": scenario,
                            "Average AUC": average_auc
                        }
                        results.append(result)

# Write the results to a CSV file
csv_file = "/storage/homefs/ch21o450/scripts/BioScenComb/validation/individual_auc_scores.csv"

with open(csv_file, mode="w", newline="") as file:
    fieldnames = ["Taxa", "Model", "Time", "Model Name", "Scenario", "Average AUC"]
    writer = csv.DictWriter(file, fieldnames=fieldnames)

    writer.writeheader()
    writer.writerows(results)

print("Results saved to", csv_file)

# Print the average AUC scores for each combination of taxa, model, time, model_name, and scenario
for result in results:
    print("Taxa:", result["Taxa"])
    print("Model:", result["Model"])
    print("Time:", result["Time"])
    print("Model Name:", result["Model Name"])
    print("Scenario:", result["Scenario"])
    print("Average AUC:", result["Average AUC"])
    print()

# Calculate overall average AUC score
average_auc_scores = [result["Average AUC"] for result in results]
overall_avg_auc_score = np.mean(average_auc_scores)
print("Overall Average AUC Score:", overall_avg_auc_score)
                       
